# 영화 정보 수집

- 1위부터 200위까지 전반적인 영화 정보를 파악하기 위한 크롤링
- 출처 : https://www.kobis.or.kr/kobis/business/stat/boxs/findFormerBoxOfficeList.do

## 1. 크롤링


    영화 정보=pd.Dataframe(
    columns=["영화제목", "개봉일", "상영타입", "장르", "등급분류(시청가능나이)", "제작국가", "감독", "주연", "관람객수"])


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:\Windows\Fonts\\malgun.ttf").get_name()
plt.rc("font", family=font_name)

import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"] = False

from tqdm.notebook import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
import re

In [4]:
## 수집해야할 리스트 ##
Title = []
Release = []
Type = []
Genre = []
Age = []
Nation = []
Director = []
Main_actor = []
Audi_list = []

In [5]:
driver = webdriver.Chrome("chromedriver")
url = "https://www.kobis.or.kr/kobis/business/stat/boxs/findFormerBoxOfficeList.do"
driver.get(url)
time.sleep(5)

try:
    ## 제목 수집 ##
    titles = driver.find_elements(By.ID, "td_movie")
    Title = [ titles[i].text for i in range(len(titles)) ]
    ## 개봉일 수집 ##
    Releases = driver.find_elements(By.ID, "td_openDt")
    Release = [ Releases[i].text for i in range(len(Releases)) ]

    for i in tqdm(range(len(Title))):
    
        ## 영화 제목 클릭 ##
        titles = driver.find_elements(By.XPATH, "//*[@id='td_movie']/span/a")
        titles[i].click()
        time.sleep(5)

        ## 요약정보 중 장르, 관람등급, 국가 수집 ##
        info = driver.find_elements(By.TAG_NAME, "dd")[3].text.replace(" ", "").split("|")
        Genre.append(info[2])
        Age.append(info[4])
        Nation.append(info[5])
        
        ## 상영 타입 수집 ##
        info = driver.find_elements(By.CLASS_NAME, "cont")
        tt = info[1].find_elements(By.TAG_NAME, "dd")[-1]
        
        ## 감독 수집 ##
        code = driver.find_elements(By.TAG_NAME, "dd")[0].text
        a = driver.find_element(By.XPATH, "//*[@id='{}_director']".format(code))
        names = a.find_element(By.TAG_NAME, "dd").text.split("|")
        Director.append(names)
        
        ## 주연 수집 ##
        time.sleep(1)
        n = driver.find_elements(By.TAG_NAME, "table")[1]
        name_list = []
        for i in n.find_elements(By.TAG_NAME, "a"):
            name_list.append(i.text)
        Main_actor.append(name_list)
        
        ## 상영타입 수집 ##
        p = re.compile("([\w\s]+) :")
        type_list = []
        
        info = driver.find_elements(By.CLASS_NAME, "cont")
        tt = info[1].find_elements(By.TAG_NAME, "dd")[-1]

        for i in tt.text.split("\n") :
            type_list.extend(p.findall(i))    
        Type.append(type_list)
        
        ## 뒤로가기 ##
        driver.find_element(By.CLASS_NAME, "close").click()
        time.sleep(1)   


except Exception as e:
    print(e)
    
finally:
    driver.close()
    driver.quit()

  0%|          | 0/200 [00:00<?, ?it/s]

In [6]:
movie_info = pd.DataFrame({"Title" : Title, "Release" : Release, "Genre" : Genre, 
                           "Age" : Age, "Nation" : Nation, "Type" : Type, 
                           "Director" : Director, "Main_actor" : Main_actor})

In [7]:
movie_info.head(10)

,Title,Release,Genre,Age,Nation,Type,Director,Main_actor
0,명량,2014-07-30,"사극,액션",15세이상관람가,한국,"[필름, 2D, 4D]",[김한민],"[최민식(이순신 ), 류승룡(구루지마 ), 조진웅(와키자카 )]"
1,극한직업,2019-01-23,코미디,15세이상관람가,한국,[2D],[이병헌],"[류승룡(고반장), 이하늬(장형사), 진선규(마형사), 이동휘(영호), 공명(재훈)]"
2,신과함께-죄와 벌,2017-12-20,"판타지,드라마",12세이상관람가,한국,"[2D, 4D]",[김용화],"[하정우(강림), 차태현(자홍), 주지훈(해원맥), 김향기(덕춘), 마동석(성주신)]"
3,국제시장,2014-12-17,드라마,12세이상관람가,한국,"[필름, 2D]",[윤제균],"[황정민(덕수), 김윤진(영자), 오달수(달구), 정진영(덕수부)]"
4,어벤져스: 엔드게임,2019-04-24,"액션,SF",12세이상관람가,미국,"[2D, 3D, 4D, IMAX]","[안소니 루소, 조 루소]","[로버트 다우니 주니어, 크리스 에반스, 마크 러팔로, 크리스 헴스워스, 스칼렛 요..."
5,겨울왕국 2,2019-11-21,애니메이션,전체관람가,미국,"[2D, 3D, 4D, IMAX]","[크리스 벅, 제니퍼 리]","[크리스틴 벨, 이디나 멘젤, 조시 게드, 조나단 그로프]"
6,베테랑,2015-08-05,"액션,드라마",15세이상관람가,한국,"[필름, 2D]",[류승완],"[황정민(서도철), 유아인(조태오), 유해진(최 상무), 오달수(오 팀장)]"
7,아바타,2009-12-17,"SF,액션,어드벤처",12세이상관람가,미국,"[필름, 2D, 3D, 4D, IMAX]",[제임스 카메론],"[샘 워싱턴, 조 샐다나, 시고니 위버]"
8,도둑들,2012-07-25,"액션,범죄",15세이상관람가,"한국,홍콩","[필름, 2D]",[최동훈],"[김윤석(마카오박), 김혜수(팹시), 이정재(뽀빠이), 전지현(예니콜), 임달화(첸..."
9,7번방의 선물,2013-01-23,"코미디,드라마",15세이상관람가,한국,"[필름, 2D]",[이환경],"[류승룡(용구), 박신혜(큰 예승), 갈소원(어린 예승), 오달수(방장), 박원상(..."


In [14]:
## save ##
movie_info.to_csv("movie_info.csv", encoding="utf-8", index=False)

## 2. 전처리

- 리스트로 들어간 값을 문자열로 저장
- Main_actor에서 () 제거
- 김도현 팀원의 파일병 변경 작업을 위해 : 을 - 로 변경

In [2]:
movie_info = pd.read_csv("movie_info.csv")

In [18]:
## 리스트로 들어간 값을 문자열로 저장 ##
index = 0
for tp in movie_info["Type"]:
    result = ','.join(s for s in tp)
    movie_info["Type"][index] = result
    index+=1

index = 0
for tp in movie_info["Director"]:
    result = ','.join(s for s in tp)
    movie_info["Director"][index] = result
    index+=1

index = 0
for tp in movie_info["Main_actor"]:
    result = ','.join(s for s in tp)
    movie_info["Main_actor"][index] = result
    index+=1

In [7]:
## Main_actor에서 () 없애기 ##
index = 0
for i in movie_info["Main_actor"]:
    actors = re.sub("\([^)]*\)", "", i)
    movie_info["Main_actor"][index] = actors
    index+=1

In [4]:
## : 을 - 로 바꾸기 ##
idx = 0
for title in movie_info["Title"]:
    movie_info["Title"][idx] = re.sub(':|{ :}|{: }',"-", title)
    idx += 1

C:\Users\eunga\AppData\Local\Temp\ipykernel_4368\737720838.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_info["Title"][idx] = re.sub(':|{ :}|{: }',"-", title)


In [5]:
movie_info

,Title,Release,Genre,Age,Nation,Type,Director,Main_actor,Audi
0,명량,2014-07-30,"사극,액션",15세이상관람가,한국,"필름,2D,4D",김한민,"최민식,류승룡,조진웅",17615919
1,극한직업,2019-01-23,코미디,15세이상관람가,한국,2D,이병헌,"류승룡,이하늬,진선규,이동휘,공명",16266338
2,신과함께-죄와 벌,2017-12-20,"판타지,드라마",12세이상관람가,한국,"2D,4D",김용화,"하정우,차태현,주지훈,김향기,마동석",14414658
3,국제시장,2014-12-17,드라마,12세이상관람가,한국,"필름,2D",윤제균,"황정민,김윤진,오달수,정진영",14264478
4,어벤져스- 엔드게임,2019-04-24,"액션,SF",12세이상관람가,미국,"2D,3D,4D,IMAX","안소니 루소,조 루소","로버트 다우니 주니어,크리스 에반스,마크 러팔로,크리스 헴스워스,스칼렛 요한슨,제레...",13977602
...,...,...,...,...,...,...,...,...,...
197,해리 포터와 불의 잔,2005-12-01,"액션,판타지",12세관람가,미국,"필름,4D",마이크 뉴웰,"다니엘 래드클리프,루퍼트 그린트,엠마 왓슨,랄프 파인즈",3499257
198,비긴 어게인,2014-08-13,멜로/로맨스,15세이상관람가,미국,"필름,2D",존 카니,"키이라 나이틀리,마크 러팔로,애덤 리바인,헤일리 스테인펠드,제임스 코든,캐서린 키너",3477672
199,태풍,2005-12-14,"드라마,액션",15세관람가,한국,"필름,2D",곽경택,"이정재,이미연,장동건",3471150
200,킹콩,2005-12-14,"드라마,어드벤처",15세관람가,미국,필름,피터 잭슨,"잭 블랙,애드리언 브로디,나오미 왓츠",3470059


In [6]:
## save ##
movie_info.to_csv("movie_info.csv", encoding="utf-8", index=False)

- 총 관객수 컬럼 추가

In [ ]:
movie = pd.read_csv("movie_info.csv")

In [3]:
## 크롤링을 통해 관람객 수 수집 추가##
driver = webdriver.Chrome("chromedriver")
url = "https://www.kobis.or.kr/kobis/business/stat/boxs/findFormerBoxOfficeList.do"
driver.get(url)
time.sleep(5)

Audi = driver.find_elements(By.ID, "td_audiAcc")
Audi_list = [ Audi[i].text for i in range(len(Audi)) ]

In [16]:
## 문자열로 가져와진 관객수를 숫자로 변환하기 위해 , 제거##
idx = 0
for i in movie["Audi"]:
    movie["Audi"][idx] = movie["Audi"][idx].replace(",", "")
    idx += 1

In [ ]:
## 숫자형으로 변환 ##
movie["Audi"].to_numeric()

In [17]:
movie

,Title,Release,Genre,Age,Nation,Type,Director,Main_actor,Audi
0,명량,2014-07-30,"사극,액션",15세이상관람가,한국,"필름,2D,4D",김한민,"최민식,류승룡,조진웅",17615919
1,극한직업,2019-01-23,코미디,15세이상관람가,한국,2D,이병헌,"류승룡,이하늬,진선규,이동휘,공명",16266338
2,신과함께-죄와 벌,2017-12-20,"판타지,드라마",12세이상관람가,한국,"2D,4D",김용화,"하정우,차태현,주지훈,김향기,마동석",14414658
3,국제시장,2014-12-17,드라마,12세이상관람가,한국,"필름,2D",윤제균,"황정민,김윤진,오달수,정진영",14264478
4,어벤져스: 엔드게임,2019-04-24,"액션,SF",12세이상관람가,미국,"2D,3D,4D,IMAX","안소니 루소,조 루소","로버트 다우니 주니어,크리스 에반스,마크 러팔로,크리스 헴스워스,스칼렛 요한슨,제레...",13977602
...,...,...,...,...,...,...,...,...,...
195,해리 포터와 불의 잔,2005-12-01,"액션,판타지",12세관람가,미국,"필름,4D",마이크 뉴웰,"다니엘 래드클리프,루퍼트 그린트,엠마 왓슨,랄프 파인즈",3499257
196,비긴 어게인,2014-08-13,멜로/로맨스,15세이상관람가,미국,"필름,2D",존 카니,"키이라 나이틀리,마크 러팔로,애덤 리바인,헤일리 스테인펠드,제임스 코든,캐서린 키너",3477672
197,태풍,2005-12-14,"드라마,액션",15세관람가,한국,"필름,2D",곽경택,"이정재,이미연,장동건",3471150
198,킹콩,2005-12-14,"드라마,어드벤처",15세관람가,미국,필름,피터 잭슨,"잭 블랙,애드리언 브로디,나오미 왓츠",3470059


In [ ]:
## save ##
movie.to_csv("movie_info_final.csv", encoding="utf-8", index=False)